In [1]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
# Opening Database:
DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
# Filtering:
craters_cat = CatalogSearch(DB, lat_bounds=[-1.5, 1.5], lon_bounds=[-129.802-1.5,-129.802+1.5], CAT_NAME='ROBBINS')

km2deg = 1/deg2km
craters_cat = craters_cat[(craters_cat.Diam < 40)&(craters_cat.Diam > 2.5)]
craters_cat['Diam']*=0.5*km2deg # km --- > deg

craters_cat_m = np.array(craters_cat)
t1 = time.time()
triplets_cat_m = find_all_triplets(craters_cat_m)
triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1','r1','lon2','lat2','r2','lon3','lat3','r3'])
t2 = time.time()
print(f'Number of total combinations:{triplets_cat.shape[0]}\nComputational time: {t2-t1:.2f} s')

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of total combinations:2730
Computational time: 0.08 s


In [11]:
# Loading All Images:
dict = load_all_images(dt=10)
# Img:
idx = 34  # Loading image n. idx+1 ...
filename = dict[str(idx+1)]
img=cv2.imread(filename)
# Detection:
t1 = time.time()
craters_det = detect(img)
t2 = time.time()
print(f'Detection Time:{t2-t1:.2f}\n')
# Pandas DataFrame:
df_craters_det = sort_mat(craters_det)
# Find all triplets:
t1 = time.time()
triplets = find_all_triplets(craters_det)
triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
triplets_det.shape
t2 = time.time()
print(f'Number of total combinations:{triplets_det.shape[0]}\nComputational time: {t2-t1:.2f} s')

Detection Time:11.17

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Number of total combinations:2730
Computational time: 0.12 s


In [24]:
t1 = time.time()
QUERY1 = triplets_cat
QUERY2 = triplets_det
QUERY1 = dropduplicates(QUERY1)
QUERY2 = dropduplicates(QUERY2) 
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
joins, items = inner_join(QUERY1, QUERY2, 2)
t2 = time.time()
print(f'Computational time: {t2-t1:.2f} s\nPossible Combinations: {len(items)}')

Computational time: 2.32 s
Possible Combinations:263


In [55]:
joins[0]

,Angle1,Angle2,Angle3,des1,des2,des3,x1,y1,r1,x2,y2,r2,x3,y3,r3
60,14.710069,49.216005,116.073925,2.071059,14.310081,33.109936,755.0,606.0,105.0,810.5,370.5,24.00,51.5,766.0,15.75
319,16.349672,49.410698,114.239630,12.927920,8.302968,5.158217,328.5,604.5,15.0,51.5,766.0,15.75,145.0,798.5,15.75


In [58]:
items[0]

Angle1     15.090486
Angle2     48.703703
Angle3    116.205811
des1        1.146015
des2        6.062466
des3        8.630192
lon1     -128.726000
lat1        0.223176
r1          0.563529
lon2     -129.290000
lat2        0.912472
r2          0.044182
lon3     -129.162000
lat3        1.193300
r3          0.049760
Name: 1, dtype: float64

In [59]:
t1 = time.time()
tol2 = 1
S, iss = [], []
for i in range(len(joins)):
    join = joins[i]
    des1, des2, des3 = items[i].des1, items[i].des2, items[i].des3
    s=join[ (abs(join.des1 - des1) < tol2) & (abs(join.des2 - des2) < tol2) & (abs(join.des3 - des3) < tol2)\
          | (abs(join.des1 - des2) < tol2) & (abs(join.des2 - des3) < tol2) & (abs(join.des3 - des1) < tol2)\
          | (abs(join.des1 - des3) < tol2) & (abs(join.des2 - des1) < tol2) & (abs(join.des3 - des2) < tol2)]

    if s.shape[0] > 0:
        S.append(s)
        iss.append(i)
t2 = time.time()
print(f'Computational time: {t2-t1:.2f} s\nPossible Combinations: {len(S)}')

Computational time: 1.30 s
Possible Combinations: 1


In [54]:
print('Combinations are:')
S[0]

Combinations are:


,Angle1,Angle2,Angle3,des1,des2,des3,x1,y1,r1,x2,y2,r2,x3,y3,r3
4,18.155007,66.879843,94.96515,2.785403,1.849742,9.740244,755.0,606.0,105.0,316.5,460.5,96.5,328.5,604.5,15.0


In [65]:
print('With:')
items[iss[0]]

With:


Angle1     17.578947
Angle2     66.066954
Angle3     96.354098
des1        2.482817
des2        1.899604
des3        9.276361
lon1     -128.486000
lat1       -0.664158
r1          0.465661
lon2     -130.240000
lat2       -0.148500
r2          0.371211
lon3     -130.167000
lat3       -0.699257
r3          0.061497
Name: 172, dtype: float64

In [ ]:
if len(joins)>1:
    print(len(joins))
    plt.close()
    i = 6
    arg = items[i][0]
    a = QUERY1.iloc[arg]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    
    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)
    plt.show()

    join = joins[i]
    j=1
    b = join.iloc[j]
    crts2 = np.vstack([[b.lon1,b.lat1,b.r1],[b.lon2,b.lat2,b.r2],[b.lon3,b.lat3,b.r3]])
    crts2[:,2]*= deg2km
    plt.subplot(122)
    lat_bounds=[-1.5, 1.5]
    lon_bounds=[-129.802-1.5,-129.802+1.5]
    crts2_to_rel = craters_to_relative_frame(crts2, lon_b=lon_bounds, lat_b=lat_bounds)
    IMG2 =  img_plus_crts(cp1, crts2_to_rel, color="green")
    plt.imshow(IMG2)
    plt.show()

In [ ]:
c2 = compute_centroid(crts2)
c1 = compute_centroid(crts1)

In [ ]:
# FIG.2
cp1 = deepcopy(img)
lat_bounds=[-1.5, 1.5]
lon_bounds=[-129.802-1.5,-129.802+1.5]
DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='ROBBINS')
df = df[df.Diam > 2.5]
image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, cp1, u=None)

plt.close()
plt.ion()
plt.figure('Image with craters catalogued', dpi=200)
plt.imshow(image_with_craters)
plt.show()

In [ ]:
cp1 = deepcopy(img)
img_det = img_plus_crts(img, craters_det)
plt.figure()
plt.imshow(img_det)
plt.show()

In [ ]:
def craters_to_relative_frame(df, lon_b, lat_b, u=None):
    lon_bounds = lon_b
    lat_bounds = lat_b
    # CAMERA CENTER:
    CAMx, CAMy = (
        (lon_bounds[0] + lon_bounds[1]) / 2,
        (lat_bounds[0] + lat_bounds[1]) / 2,
    )

    if u == None:  # Scale Factor
        u = 257.52  # ? DEG TO PXS
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
        print(span)
    else:
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
    # Make the img:
    # img = np.zeros((span, span), dtype=int)
    if df is None:
        print("No crater found")
        pass
    else:
        W, H = (span, span)
        # Cycle through the dataframe:
        craters = np.zeros(3)
        for i in range(df.shape[0]):
            crater = df[i]
            # crater center:
            xc, yc = crater[0], crater[1]  # This is in the absolute frame
            # f: Absolute --> f: Relative
            xc = xc - CAMx
            yc = yc - CAMy
            # f: relative --> f: OPENCV
            xc *= u  # Now is in pixel not in lon deg
            yc *= u  # Now is in pixel not in lat deg
            xc = W / 2 + xc
            yc = H / 2 - yc
            # ? 1 km = 8.4746 px in our DEM := Merge LOLA - KAGUYA
            KM_to_PX = 1/0.118
            crater_i = [xc, yc, crater[2] * KM_to_PX]
            craters = np.vstack([craters, crater_i])
        return craters[1:, :]